In [34]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold
from sklearn import metrics
from imblearn.over_sampling import SMOTE 

In [35]:
train = pd.read_csv("C:/Users/user/Documents/likely to default training data/training_data_29_02_2024.csv")

In [36]:
train = train.dropna()

In [37]:
train.head()

,accountId,end_of_month,paymentSequence,repaid,in_default_30,in_default_60
0,2,8/31/2021 12:00:00 AM,1,1.0,0,0
1,2,9/30/2021 12:00:00 AM,2,1.0,0,0
2,2,10/31/2021 12:00:00 AM,3,1.0,0,0
3,2,11/30/2021 12:00:00 AM,4,0.8,0,0
4,2,12/31/2021 12:00:00 AM,5,0.8,0,0


In [38]:
train.in_default_30.value_counts()

in_default_30
0    290374
1    116270
Name: count, dtype: int64

In [39]:
train["repaid"].astype("category")

0         1.0
1         1.0
2         1.0
3         0.8
4         0.8
         ... 
406639    1.0
406640    1.0
406641    1.0
406642    1.0
406643    1.0
Name: repaid, Length: 406644, dtype: category
Categories (24, float64): [0.0, 0.1, 0.2, 0.3, ..., 2.0, 2.2, 3.0, 4.0]

In [40]:
X = train.iloc[:,[3]]
print(X)

        repaid
0          1.0
1          1.0
2          1.0
3          0.8
4          0.8
...        ...
406639     1.0
406640     1.0
406641     1.0
406642     1.0
406643     1.0

[406644 rows x 1 columns]


In [41]:
y = train.iloc[:,[4]]
print(y)

        in_default_30
0                   0
1                   0
2                   0
3                   0
4                   0
...               ...
406639              0
406640              0
406641              0
406642              0
406643              0

[406644 rows x 1 columns]


In [42]:
os = SMOTE(random_state = 0)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.25, random_state = 100)
columns = X_train.columns

In [43]:
smote_data_X, smote_data_y = os.fit_resample(X_train,y_train)

In [44]:
smote_data_X = pd.DataFrame(data=smote_data_X, columns=columns)

In [45]:
smote_data_y = pd.DataFrame(data=smote_data_y, columns=["in_default_30"])

In [46]:
#params = {
#    'n_estimators': [100, 200, 500],
#    'learning_rate': [0.01,0.05,0.1],
#    'booster': ['gbtree', 'gblinear'],
#    'gamma': [0, 0.5, 1],
#    'reg_alpha': [0, 0.5, 1],
#    'reg_lambda': [0.5, 1, 5],
#    'base_score': [0.2, 0.5, 1]
#}

In [47]:
Xgb = XGBClassifier()
#Gs =  GridSearchCV(Xgb, params, n_jobs=-1, cv=KFold(n_splits=3), scoring='precision')

In [48]:
model = Xgb.fit(smote_data_X, smote_data_y.values.ravel())

In [49]:
#print('Best score:', model.best_score_)
#print('Best score:', model.best_params_)

In [50]:
y_predicted = model.predict(X_test)

In [52]:
print(confusion_matrix(y_test,y_predicted))
print(classification_report(y_test,y_predicted))
print(roc_auc_score(y_test,y_predicted))

[[64156  8436]
 [    8 29061]]
              precision    recall  f1-score   support

           0       1.00      0.88      0.94     72592
           1       0.78      1.00      0.87     29069

    accuracy                           0.92    101661
   macro avg       0.89      0.94      0.91    101661
weighted avg       0.94      0.92      0.92    101661

0.941756819995212


In [60]:
test = pd.ExcelFile("C:/Users/user/Downloads/probability of default (1).xlsx").parse("probability of default")

In [61]:
test = test.dropna()

In [62]:
test.head()

,accountId,account_ref,customer_names,phone_number,product,status,end_of_month,paymentSequence,repaid
0,6,29863318,John Oluoch Oguto,254717459831,RainMaker 2 + TV,Current,2024-03-31,32,0.968967
1,61,21983558,Silas Kariuki,254727431365,CS + TV,Arrears,2024-03-31,23,0.869565
2,66,27922697-1,Gilbert Rono,254717895246,RainMaker 2 + TV,Current,2024-03-31,26,0.961538
3,87,29952002,Oscar Kagari,254717436446,CS + TV,Current,2024-03-31,23,0.962799
4,105,36319526,Nathan kirui,254706839511,RainMaker 2.0,Current,2024-03-31,21,0.990909


In [64]:
test["repaid"].round(1).astype("category")

0        1.0
1        0.9
2        1.0
3        1.0
4        1.0
        ... 
10411    0.0
10412    0.0
10416    0.1
10417    0.0
10418    0.0
Name: repaid, Length: 10114, dtype: category
Categories (18, float64): [0.0, 0.1, 0.2, 0.3, ..., 1.4, 1.6, 1.8, 2.0]

In [65]:
X_new = test.iloc[:,[8]]

In [66]:
Y_predict = model.predict(X_new)

In [67]:
Y_predict_prob = model.predict_proba(X_new)

In [69]:
#test["missing_payment_result"] = Y_predict.tolist()
test["default_result_prob"] = Y_predict_prob.tolist()
test["default_result"] = Y_predict.tolist()

In [70]:
test.to_csv("C:/Users/user/Documents/likely to default training data/results_29_02_24.csv")